# 文件与IO

#### 字符串作为文件对象进行I/O操作

In [1]:
import io
s = io.StringIO()
s.write('Hello World\n')
print('This is a test', file=s)
s.getvalue()

'Hello World\nThis is a test\n'

In [2]:
s = io.StringIO('Hello\nWorld\n')
s.read(4)

'Hell'

In [3]:
s.read()

'o\nWorld\n'

In [4]:
s = io.BytesIO()
s.write(b'binary data')
s.getvalue()

b'binary data'

#### 固定大小记录的文件迭代

In [5]:
# 用固定长度记录或者数据块的序列，取代一行一行的迭代
from functools import partial

RECORD_SIZE = 20

with open('文件与IO.ipynb', 'rb') as f:
    records = iter(partial(f.read, RECORD_SIZE), b'')
    counter = 0
    for r in records:
        print(r)
        counter += 1
        if counter > 5:
            break

b'{\n "cells": [\n  {\n  '
b' "cell_type": "markd'
b'own",\n   "metadata":'
b' {},\n   "source": [\n'
b'    "# \xe6\x96\x87\xe4\xbb\xb6\xe4\xb8\x8eIO"\n'
b'   ]\n  },\n  {\n   "ce'


#### 读取二进制数据到可变缓冲区中

In [6]:
import os.path

def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

In [7]:
read_into_buffer('文件与IO.ipynb')[:50]

bytearray(b'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "')

#### 内存映射文件

In [8]:
"""
mmap 模块可以将文件映射到内存地址空间，
并且不是将全文件加载到内存，而是只加载访问部分，
这样既不占用过多内存，还拥有内存级别的IO速度，
适合处理超大文件的读写
"""

import os
import mmap

def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

In [9]:
m = memory_map('文件与IO.ipynb')

In [10]:
len(m)

10836

In [11]:
m[0:50]

b'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "'

#### 获取文件夹中的文件列表

In [12]:
import os
# 返回目录下所有文件的列表
names = os.listdir('./')

# glob 返回匹配文件的列表
import glob
files1 = glob.glob('文件与IO.*')

# fnmatch 返回是否存在匹配的布尔值
from fnmatch import fnmatch
files2 = [name for name in names 
          if fnmatch(name, '*.ipynb')]

In [13]:
files1, files2

(['文件与IO.ipynb'],
 ['数字日期和时间.ipynb',
  '文件与IO.ipynb',
  '迭代器与生成器.ipynb',
  '数据结构和算法.ipynb',
  '字符串和文本.ipynb'])

#### 打印不合法的文件名

In [14]:
"""
surrogateescape:
是Python在绝大部分面向OS API中所使用的错误处理器，
它能以一种优雅的方式处理由操作系统提供的数据的编码问题。
在解码出错时会将出错字节存储到一个很少被使用到的Unicode编码范围内，
在编码时将那些隐藏值又还原回原先解码失败的字节序列。
它不仅对于OS API非常有用，也能很容易的处理其他情况下的编码错误。
"""

def bad_filename(filename):
    temp = filename.encode(sys.getfilesystemencoding(), 
                           errors='surrogateescape')
    return temp.decode('latin-1')

In [15]:
files = os.listdir('.')

for name in files:
    try:
        print(name)
    except UnicodeEncodeError:
        print(bad_filename(name))

数字日期和时间.ipynb
文件与IO.ipynb
.gitignore
迭代器与生成器.ipynb
数据结构和算法.ipynb
.ipynb_checkpoints
字符串和文本.ipynb
.git


#### 创建临时文件和文件夹

In [16]:
# 创建临时文件
from tempfile import TemporaryFile

# 该文件没有文件名和目录，关闭后自动删除
with TemporaryFile('w+t') as f:
    f.write('Hello World\n')
    f.write('Testing\n')
    f.seek(0)
    data = f.read()
    print(data)

Hello World
Testing



In [17]:
from tempfile import NamedTemporaryFile

# 使用这个函数代替，并使用 delete=False 参数，可以避免被匿名和删除
with NamedTemporaryFile('w+t', delete=False) as f:
    print('filename is:', f.name)

filename is: /var/folders/jy/8jjh9yq90rbgvsw6__r5sbnw0000gn/T/tmp93dkmf8p


In [18]:
# 创建临时文件夹
from tempfile import TemporaryDirectory

with TemporaryDirectory() as dirname:
    print('dirname is:', dirname)

dirname is: /var/folders/jy/8jjh9yq90rbgvsw6__r5sbnw0000gn/T/tmp52asc5i0


In [19]:
# 临时文件在系统默认的位置被创建，该方法可以获得位置
import tempfile
tempfile.gettempdir()

'/var/folders/jy/8jjh9yq90rbgvsw6__r5sbnw0000gn/T'